In [1]:
import sys

sys.path.insert(0, '..')

In [2]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
import torch.nn.functional as F
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction, TimeSeriesTransformerModel 
from transformers.models.time_series_transformer.modeling_time_series_transformer import TimeSeriesTransformerEncoder
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from scipy import stats
import json 
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime

from pathlib import Path
from core.multimodal.dataset import collate_fn, ASASSNVarStarDataset
from functools import partial
import matplotlib.pyplot as plt

/home/mrizhko/anaconda3/envs/period/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
"""
We used 16 features from Gaia EDR3, 2MASS and AllWISE. These
include the EDR3 G, BP, RP magnitudes and the associated uncertainties, the 𝐵𝑃−𝑅𝑃 color, the 𝐵𝑃−𝑅𝑃 excess factor, signal-to-noise
ratios in G and BP, the renormalized unit weight error (RUWE), the
𝐽 − 𝐾𝑠 color, the absolute 𝑊𝑅𝑃 magnitude and the absolute 𝑊𝐽 𝐾
magnitude. The EDR3 signal-to-noise ratios are essentially the ratio of the observed flux divided by the error in the flux. As noted
earlier, the EDR3 photometric uncertainties and flux errors encode
information about the photometric variability of stars. We also used
the absolute, “reddening-free” Wesenheit magnitudes (Madore 1982;
Lebzelter et al. 2018)
𝑊𝑅𝑃 = 𝑀RP − 1.3(𝐵𝑃 − 𝑅𝑃) , (1)
and
𝑊𝐽 𝐾 = 𝑀Ks − 0.686(𝐽 − 𝐾𝑠) (2)
and the probabilistic EDR3 distances from Bailer-Jones et al. (2021)
"""

'\nWe used 16 features from Gaia EDR3, 2MASS and AllWISE. These\ninclude the EDR3 G, BP, RP magnitudes and the associated uncertainties, the 𝐵𝑃−𝑅𝑃 color, the 𝐵𝑃−𝑅𝑃 excess factor, signal-to-noise\nratios in G and BP, the renormalized unit weight error (RUWE), the\n𝐽 − 𝐾𝑠 color, the absolute 𝑊𝑅𝑃 magnitude and the absolute 𝑊𝐽 𝐾\nmagnitude. The EDR3 signal-to-noise ratios are essentially the ratio of the observed flux divided by the error in the flux. As noted\nearlier, the EDR3 photometric uncertainties and flux errors encode\ninformation about the photometric variability of stars. We also used\nthe absolute, “reddening-free” Wesenheit magnitudes (Madore 1982;\nLebzelter et al. 2018)\n𝑊𝑅𝑃 = 𝑀RP − 1.3(𝐵𝑃 − 𝑅𝑃) , (1)\nand\n𝑊𝐽 𝐾 = 𝑀Ks − 0.686(𝐽 − 𝐾𝑠) (2)\nand the probabilistic EDR3 distances from Bailer-Jones et al. (2021)\n'